# 03 - Multi Layer Network

In [1]:
import tools.processing as pre
import re

# use less text for now to avoid memory error
text = pre.get_text("data/cleaned-rap-lyrics/clean2_pac_.txt")[:20002]
vocab = pre.Vocabulary(text)
# double \\n to avoid null error in tensorboard projection
text = text.replace("\n", " \\n ")
# remove extra spacing
tokens = re.sub( " +", " ", text).split(" ")[:-1]

TIMESTEPS = 10

str_data, str_labels = pre.create_data_label_pairs(tokens, TIMESTEPS)

print( list( zip(str_data, str_labels) )[:5] )

[(['as', 'real', 'as', 'it', 'seems', 'the', 'american', 'dream', '\\n', "ain't"], 'nothing'), (['real', 'as', 'it', 'seems', 'the', 'american', 'dream', '\\n', "ain't", 'nothing'], 'but'), (['as', 'it', 'seems', 'the', 'american', 'dream', '\\n', "ain't", 'nothing', 'but'], 'another'), (['it', 'seems', 'the', 'american', 'dream', '\\n', "ain't", 'nothing', 'but', 'another'], 'calculated'), (['seems', 'the', 'american', 'dream', '\\n', "ain't", 'nothing', 'but', 'another', 'calculated'], 'schemes')]


In [2]:
import tools.training as tr

encoder = tr.OneHotWordEncoder("1-Hot-Word-Encoding", vocab.word2index)
decoder = tr.OneHotWordDecoder("1-Hot-Word-Decoding", vocab.index2word, temperature=0.8)

data = encoder.encode( str_data )
labels = encoder.encode_labels( str_labels )

In [3]:
import tools.processing as pre
import tools.architectures as nn

# text = pre.get_text("data/cleaned-rap-lyrics/cleanrakim_.txt")

# vocab = pre.Vocabulary(text)

NUM_LAYERS = 2
HIDDEN_LAYER_SIZE = 512
VOCAB_SIZE = vocab.get_size()
TIMESTEPS = 10

EPOCHS = 20
BATCH_SIZE = 256

# data, labels = vocab.making_one_hot(text, TIMESTEPS)

rnn = nn.MultiLayerRNN(name = "multi-rakim")
rnn.build(NUM_LAYERS, HIDDEN_LAYER_SIZE, VOCAB_SIZE, TIMESTEPS, l2_reg=0.0)

sampler = lambda trainable, seed_text: tr.sample( seed_text, trainable, encoder, decoder, length=20)

tr.train_model(rnn, data, labels, sampler, epochs=EPOCHS, batch_size=BATCH_SIZE)



Epoch 1/20
Loss:    	 5.718593120574951
Accuracy:	 10.663984298706055
------Sampling----------
seed: 	as real as it seems the american dream
ain't nothing but another calculated schemes
to get us locked up
result:as real as it seems the american dream
ain't nothing but another calculated schemes
to get us locked up scared so the 
 impatient will now like but my it i the a do aiyyo damn 
 
 i


Epoch 2/20
Loss:    	 5.613383769989014
Accuracy:	 10.663984298706055
------Sampling----------
seed: 	as real as it seems the american dream
ain't nothing but another calculated schemes
to get us locked up
result:as real as it seems the american dream
ain't nothing but another calculated schemes
to get us locked up i i'm my terror put 
 
 
 i to the like could could life to you'll but my i


Epoch 3/20
Loss:    	 5.551100254058838
Accuracy:	 10.663984298706055
------Sampling----------
seed: 	as real as it seems the american dream
ain't nothing but another calculated schemes
to get us locked up


In [6]:
sampler(rnn, "while i go down the street \n you was lookin' at me \n is this even good \n why")

------Sampling----------
seed: 	while i go down the street 
 you was lookin' at me 
 is this even good 
 why
result:while i go down the street 
 you was lookin' at me 
 is this even good 
 why 
 that tip knew glue my 
 my heart goin' and time to word 
 you'll you you of why


# Problem

How can we work out proper features from the text?

Just because a line does not match 100% with the original one that doesn't mean that it is bad